<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=3b8fe6304ef47be8ea8fe7cb084864f7c9271e883a75e0a4fdaee0c16f07444a
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [2]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-5*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

mypf = mypf[mypf['InPortfolio'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
stock_ids

Mounted at /content/drive


array(['5PAISA', 'AAVAS', 'AMARAJABAT', 'AMBUJACEM', 'ASIANPAINT',
       'BAJAJFINSV', 'BAJFINANCE', 'BANDHANBNK', 'BATAINDIA', 'BAYERCROP',
       'CLEAN', 'DMART', 'FINEORG', 'GICRE', 'GILLETTE', 'GLAND',
       'HDFCAMC', 'ICICIGI', 'INDIGOPNTS', 'INDUSINDBK', 'INFY', 'IRCTC',
       'ITBEES', 'JMFINANCIL', 'JUBLFOOD', 'LUXIND', 'NIACL', 'PAGEIND',
       'PIDILITIND', 'PNB', 'RELAXO', 'RELIANCE', 'ROSSARI', 'TCS',
       'TEAMLEASE', 'UJJIVANSFB', 'VAIBHAVGBL', 'VALIANTORG', 'WIPRO'],
      dtype=object)

In [5]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))
    stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Stock'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'
                   ], axis=1, inplace=True)

    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    #current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'
    tmp = stock_prec_dev(stock_id)
    tmp['Prev_Close'] = tmp['Close'].shift(1)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Criteria'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]
    tmp['Strategy'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
    #tmp['Curr_PE'] = current_pe
    df_prec_dev = df_prec_dev.append(tmp)

In [6]:
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

In [7]:
# portfolio stocks

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df['Investment'] = tmp_df['Avg_Cost'] * tmp_df['Shares']
tmp_df['Current'] = tmp_df['Close'] * tmp_df['Shares']
tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
tmp_df['Estimated'] = tmp_df['Max'] * tmp_df['Shares']
tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Estimated P/L%'] = round((tmp_df['Estimated'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Juice Left%'] = tmp_df['Estimated P/L%'] - tmp_df['Current P/L%']

investment = round(sum(tmp_df['Avg_Cost']*tmp_df['Shares']),0)
current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
previous = round(sum(tmp_df['Prev_Close']*tmp_df['Shares']),0)

today_pnl_amount = current-previous
today_pnl_percentage = round((current-previous)*100/previous,2)

curr_pnl_amount = current-investment
curr_pnl_percentage = round((current-investment)*100/investment,2)

estimate = round(sum(tmp_df['Max']*tmp_df['Shares']),0)
est_pnl_amount = estimate-investment
est_pnl_percentage = round((estimate-investment)*100/investment,2)

tmp_df['Weightage%'] = round(tmp_df['Investment']*100/investment,2)
tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['Weightage%'],0)

print('Investment: ', investment)
print('Current: ', current)
print('Today PnL: ', today_pnl_amount)
print('Today PnL %: ', today_pnl_percentage)
print('Current PnL: ', curr_pnl_amount)
print('Current PnL %: ', curr_pnl_percentage)
print('Estimate: ', estimate)
print('Estimate PnL: ', est_pnl_amount)
print('Estimate PnL %: ', est_pnl_percentage)

qualified stocks: 39
Investment:  2773435.0
Current:  2862416.0
Today PnL:  29119.0
Today PnL %:  1.03
Current PnL:  88981.0
Current PnL %:  3.21
Estimate:  4990903.0
Estimate PnL:  2217468.0
Estimate PnL %:  79.95


In [8]:
tmp_df.sort_values(by = 'Current P/L%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
11,4085.60,DMART.NS,5324.0,3917.70,3671.87,21.0,SS,ATH,77109.27,85797.60,82271.70,111804.0,4.29,11.27,44.99,33.72,2.78,31.0
17,1254.75,ICICIGI.NS,1613.0,1209.50,1137.80,69.0,V40,ATH,78508.20,86577.75,83455.50,111297.0,3.74,10.28,41.76,31.48,2.83,29.0
6,7203.20,BAJFINANCE.NS,7901.0,7126.60,6602.15,24.0,V40,ATH,158451.60,172876.80,171038.40,189624.0,1.07,9.10,19.67,10.57,5.71,52.0
13,189.75,GICRE.NS,358.0,181.65,174.18,231.0,2T3Y,2T3Y,40235.58,43832.25,41961.15,82698.0,4.46,8.94,105.53,96.59,1.45,13.0
3,460.90,AMBUJACEM.NS,588.0,458.35,424.02,180.0,TBD,ATH,76323.60,82962.00,82503.00,105840.0,0.56,8.70,38.67,29.97,2.75,24.0
28,2697.70,PIDILITIND.NS,2902.0,2680.70,2504.06,30.0,V40,ATH,75121.80,80931.00,80421.00,87060.0,0.63,7.73,15.89,8.16,2.71,21.0
35,37.55,UJJIVANSFB.NS,57.0,38.10,34.87,1350.0,V40N,ATH,47074.50,50692.50,51435.00,76950.0,-1.44,7.69,63.46,55.77,1.70,13.0
4,3316.85,ASIANPAINT.NS,3542.0,3294.35,3091.34,25.0,V40,ATH,77283.50,82921.25,82358.75,88550.0,0.68,7.29,14.58,7.29,2.79,20.0
23,73.55,JMFINANCIL.NS,118.0,71.45,69.17,675.0,2T3Y,2T3Y,46689.75,49646.25,48228.75,79650.0,2.94,6.33,70.59,64.26,1.68,11.0
8,1616.10,BATAINDIA.NS,2173.0,1614.60,1523.62,100.0,V40,ATH,152362.00,161610.00,161460.00,217300.0,0.09,6.07,42.62,36.55,5.49,33.0


In [9]:
tmp_df.sort_values(by = 'Estimated P/L%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
15,1009.05,GLAND.NS,4316.0,1008.45,1225.87,55.0,2T3Y,2T3Y,67422.85,55497.75,55464.75,237380.0,0.06,-17.69,252.08,269.77,2.43,-43.0
37,558.15,VALIANTORG.NS,1851.0,547.20,542.52,144.0,2T3Y,2T3Y,78122.88,80373.60,78796.80,266544.0,2.00,2.88,241.19,238.31,2.82,8.0
25,1510.45,LUXIND.NS,4546.0,1505.80,1493.72,51.0,V40N,2T3Y,76179.72,77032.95,76795.80,231846.0,0.31,1.12,204.34,203.22,2.75,3.0
36,308.45,VAIBHAVGBL.NS,966.0,305.30,317.48,144.0,2T3Y,2T3Y,45717.12,44416.80,43963.20,139104.0,1.03,-2.84,204.27,207.11,1.65,-5.0
7,253.95,BANDHANBNK.NS,723.0,246.95,253.98,300.0,2T3Y,2T3Y,76194.00,76185.00,74085.00,216900.0,2.83,-0.01,184.67,184.68,2.75,-0.0
26,123.05,NIACL.NS,317.0,120.15,117.07,390.0,TBD,ATH,45657.30,47989.50,46858.50,123630.0,2.41,5.11,170.78,165.67,1.65,8.0
34,2387.10,TEAMLEASE.NS,5281.0,2339.15,2279.93,21.0,V40N,2T3Y,47878.53,50129.10,49122.15,110901.0,2.05,4.70,131.63,126.93,1.73,8.0
1,1353.75,AAVAS.NS,3196.0,1340.50,1390.60,23.0,2T3Y,2T3Y,31983.80,31136.25,30831.50,73508.0,0.99,-2.65,129.83,132.48,1.15,-3.0
18,1418.05,INDIGOPNTS.NS,3112.0,1405.35,1407.73,124.0,V40N,2T3Y,174558.52,175838.20,174263.40,385888.0,0.90,0.73,121.07,120.34,6.29,5.0
13,189.75,GICRE.NS,358.0,181.65,174.18,231.0,2T3Y,2T3Y,40235.58,43832.25,41961.15,82698.0,4.46,8.94,105.53,96.59,1.45,13.0


In [10]:
tmp_df.sort_values(by = 'Juice Left%', ascending=True)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
4,3316.85,ASIANPAINT.NS,3542.0,3294.35,3091.34,25.0,V40,ATH,77283.50,82921.25,82358.75,88550.0,0.68,7.29,14.58,7.29,2.79,20.0
28,2697.70,PIDILITIND.NS,2902.0,2680.70,2504.06,30.0,V40,ATH,75121.80,80931.00,80421.00,87060.0,0.63,7.73,15.89,8.16,2.71,21.0
31,2577.40,RELIANCE.NS,2811.0,2552.55,2467.15,31.0,V40,ATH,76481.65,79899.40,79129.05,87141.0,0.97,4.47,13.94,9.47,2.76,12.0
6,7203.20,BAJFINANCE.NS,7901.0,7126.60,6602.15,24.0,V40,ATH,158451.60,172876.80,171038.40,189624.0,1.07,9.10,19.67,10.57,5.71,52.0
33,3174.90,TCS.NS,3857.0,3216.30,3231.00,16.0,V40,NaN,51696.00,50798.40,51460.80,61712.0,-1.29,-1.74,19.37,21.11,1.86,-3.0
5,1508.60,BAJAJFINSV.NS,1907.0,1474.70,1454.93,52.0,V40,ATH,75656.36,78447.20,76684.40,99164.0,2.30,3.69,31.07,27.38,2.73,10.0
3,460.90,AMBUJACEM.NS,588.0,458.35,424.02,180.0,TBD,ATH,76323.60,82962.00,82503.00,105840.0,0.56,8.70,38.67,29.97,2.75,24.0
17,1254.75,ICICIGI.NS,1613.0,1209.50,1137.80,69.0,V40,ATH,78508.20,86577.75,83455.50,111297.0,3.74,10.28,41.76,31.48,2.83,29.0
22,30.03,ITBEES.NS,40.0,30.09,30.07,1675.0,SS,ATH,50367.25,50300.25,50400.75,67000.0,-0.20,-0.13,33.02,33.15,1.82,-0.0
11,4085.60,DMART.NS,5324.0,3917.70,3671.87,21.0,SS,ATH,77109.27,85797.60,82271.70,111804.0,4.29,11.27,44.99,33.72,2.78,31.0


In [11]:
tmp_df.sort_values(by = 'Weightage%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
18,1418.05,INDIGOPNTS.NS,3112.0,1405.35,1407.73,124.0,V40N,2T3Y,174558.52,175838.20,174263.40,385888.0,0.90,0.73,121.07,120.34,6.29,5.0
6,7203.20,BAJFINANCE.NS,7901.0,7126.60,6602.15,24.0,V40,ATH,158451.60,172876.80,171038.40,189624.0,1.07,9.10,19.67,10.57,5.71,52.0
8,1616.10,BATAINDIA.NS,2173.0,1614.60,1523.62,100.0,V40,ATH,152362.00,161610.00,161460.00,217300.0,0.09,6.07,42.62,36.55,5.49,33.0
27,38568.40,PAGEIND.NS,53549.0,38377.00,39325.03,3.0,V40,ATH,117975.09,115705.20,115131.00,160647.0,0.50,-1.92,36.17,38.09,4.25,-8.0
14,4540.30,GILLETTE.NS,7529.0,4513.75,4475.85,21.0,V40,ATH,93992.85,95346.30,94788.75,158109.0,0.59,1.44,68.21,66.77,3.39,5.0
0,364.60,5PAISA.NS,553.0,366.50,347.06,228.0,V40N,ATH,79129.68,83128.80,83562.00,126084.0,-0.52,5.05,59.34,54.29,2.85,14.0
17,1254.75,ICICIGI.NS,1613.0,1209.50,1137.80,69.0,V40,ATH,78508.20,86577.75,83455.50,111297.0,3.74,10.28,41.76,31.48,2.83,29.0
37,558.15,VALIANTORG.NS,1851.0,547.20,542.52,144.0,2T3Y,2T3Y,78122.88,80373.60,78796.80,266544.0,2.00,2.88,241.19,238.31,2.82,8.0
4,3316.85,ASIANPAINT.NS,3542.0,3294.35,3091.34,25.0,V40,ATH,77283.50,82921.25,82358.75,88550.0,0.68,7.29,14.58,7.29,2.79,20.0
24,498.15,JUBLFOOD.NS,903.0,497.00,471.56,164.0,SS,ATH,77335.84,81696.60,81508.00,148092.0,0.23,5.64,91.49,85.85,2.79,16.0


In [12]:
tmp_df.to_csv('/content/drive/My Drive/data/stocks/myStocks-Portfolio-Analysis.csv', index=False)

In [13]:
tmp_df.sort_values(by = 'Risk Ind', ascending=True)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
15,1009.05,GLAND.NS,4316.0,1008.45,1225.87,55.0,2T3Y,2T3Y,67422.85,55497.75,55464.75,237380.0,0.06,-17.69,252.08,269.77,2.43,-43.0
27,38568.40,PAGEIND.NS,53549.0,38377.00,39325.03,3.0,V40,ATH,117975.09,115705.20,115131.00,160647.0,0.50,-1.92,36.17,38.09,4.25,-8.0
38,381.30,WIPRO.NS,713.0,388.95,388.07,195.0,2T3Y,2T3Y,75673.65,74353.50,75845.25,139035.0,-1.97,-1.74,83.73,85.47,2.73,-5.0
36,308.45,VAIBHAVGBL.NS,966.0,305.30,317.48,144.0,2T3Y,2T3Y,45717.12,44416.80,43963.20,139104.0,1.03,-2.84,204.27,207.11,1.65,-5.0
1,1353.75,AAVAS.NS,3196.0,1340.50,1390.60,23.0,2T3Y,2T3Y,31983.80,31136.25,30831.50,73508.0,0.99,-2.65,129.83,132.48,1.15,-3.0
33,3174.90,TCS.NS,3857.0,3216.30,3231.00,16.0,V40,NaN,51696.00,50798.40,51460.80,61712.0,-1.29,-1.74,19.37,21.11,1.86,-3.0
10,1383.95,CLEAN.NS,2648.0,1363.55,1396.49,23.0,2T3Y,2T3Y,32119.27,31830.85,31361.65,60904.0,1.50,-0.90,89.62,90.52,1.16,-1.0
2,624.50,AMARAJABAT.NS,988.0,624.55,625.25,72.0,V200,ATH,45018.00,44964.00,44967.60,71136.0,-0.01,-0.12,58.02,58.14,1.62,-0.0
32,848.10,ROSSARI.NS,1569.0,844.20,844.40,36.0,2T3Y,2T3Y,30398.40,30531.60,30391.20,56484.0,0.46,0.44,85.81,85.37,1.10,0.0
7,253.95,BANDHANBNK.NS,723.0,246.95,253.98,300.0,2T3Y,2T3Y,76194.00,76185.00,74085.00,216900.0,2.83,-0.01,184.67,184.68,2.75,-0.0
